In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import rankdata, skew
from sklearn.feature_extraction.text import TfidfVectorizer
from concurrent.futures import ThreadPoolExecutor
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, QuantileTransformer
from sklearn.utils import resample
import traceback
import re
from imblearn.over_sampling import SMOTE
from collections import Counter

# Hàm để thử nhiều kiểu encoding cho đến khi đọc được file
def read_csv_with_multiple_encodings(file_path, encodings=['utf-8', 'latin1', 'ISO-8859-1', 'cp1252']):
    """
    Thử đọc file CSV với nhiều kiểu encoding khác nhau cho đến khi thành công.
    """
    for encoding in encodings:
        try:
            print(f"Đang thử đọc với encoding: {encoding}")
            df = pd.read_csv(file_path, encoding=encoding)
            print(f"Đọc thành công với encoding: {encoding}")
            return df
        except UnicodeDecodeError:
            print(f"Không thể đọc với encoding: {encoding}")
            continue
        except Exception as e:
            print(f"Lỗi khác: {str(e)}")
            continue
    
    try:
        print("Đang thử đọc với engine='python' và encoding='latin1'")
        return pd.read_csv(file_path, encoding='latin1', engine='python')
    except Exception as e:
        print(f"Tất cả các phương pháp đều thất bại: {str(e)}")
        try:
            print("Đang thử phương pháp cuối cùng: đọc file dạng nhị phân")
            import io
            with open(file_path, 'rb') as f:
                content = f.read()
                content = content.decode('latin1').encode('utf-8', 'replace').decode('utf-8')
                return pd.read_csv(io.StringIO(content))
        except Exception as final_e:
            print(f"Không thể đọc file với bất kỳ phương pháp nào: {str(final_e)}")
            raise

# ====== CLEANING FUNCTIONS ======
def clean_price(price_str):
    """Làm sạch giá từ chuỗi"""
    if pd.isna(price_str):
        return 0.0
    
    if isinstance(price_str, str):
        price_clean = re.sub(r'[^\d.]', '', price_str)
        if price_clean:
            try:
                return float(price_clean)
            except:
                return 0.0
        return 0.0
    
    try:
        return float(price_str) if pd.notna(price_str) else 0.0
    except:
        return 0.0

def clean_percentage(percentage_str):
    """Làm sạch phần trăm từ chuỗi"""
    if pd.isna(percentage_str):
        return 0.0
    
    if isinstance(percentage_str, str):
        percentage_clean = re.sub(r'[^\d.]', '', percentage_str)
        if percentage_clean:
            try:
                return float(percentage_clean) / 100
            except:
                return 0.0
        return 0.0
    
    try:
        return float(percentage_str) if pd.notna(percentage_str) else 0.0
    except:
        return 0.0

def clean_rating_count(rating_count_str):
    """Làm sạch số lượng đánh giá từ chuỗi"""
    if pd.isna(rating_count_str):
        return 0
    
    if isinstance(rating_count_str, str):
        count_clean = re.sub(r'[^\d]', '', rating_count_str)
        if count_clean:
            try:
                return int(count_clean)
            except:
                return 0
        return 0
    
    try:
        return int(rating_count_str) if pd.notna(rating_count_str) else 0
    except:
        return 0

# ====== NORMALIZATION ======
def normalize_numeric(df):
    """Chuẩn hóa các cột số"""
    numeric_columns = ['rating', 'rating_count', 'review_count', 
                       'discounted_price', 'actual_price', 'discount_percentage']
    for col in numeric_columns:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
            df[col] = df[col].clip(0)  # không cho giá trị âm
    return df

# ====== OUTLIER HANDLING ======
def handle_outliers(df, columns):
    """Xử lý ngoại lai bằng phương pháp IQR"""
    for column in columns:
        if column in df.columns:
            Q1 = df[column].quantile(0.25)
            Q3 = df[column].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            df[column] = df[column].clip(lower=lower_bound, upper=upper_bound)
    return df

# ====== ENCODING ======
def preprocess_data(df):
    """Tiền xử lý dữ liệu chính và tính trọng số cho rating"""
    # Loại bỏ các dòng thiếu user_id, product_id, hoặc rating
    df = df.dropna(subset=['user_id', 'product_id', 'rating'])
    
    # Xử lý user_id nếu chứa nhiều ID (phân tách bằng dấu phẩy)
    df['user_id'] = df['user_id'].apply(
        lambda x: x.split(',')[0].strip() if isinstance(x, str) and ',' in x else x
    )
    
    # Đảm bảo rating là số và trong phạm vi 1-5
    df['rating'] = pd.to_numeric(df['rating'], errors='coerce')
    df = df.dropna(subset=['rating'])  # Loại bỏ những dòng với rating không hợp lệ
    df['rating'] = df['rating'].clip(1, 5)
    
    # Chuẩn hóa rating để cải thiện sự ổn định trong huấn luyện
    scaler = MinMaxScaler(feature_range=(0, 1))
    df['scaled_rating'] = scaler.fit_transform(df[['rating']])
    
    # Mã hóa user_id và product_id thành chỉ số số nguyên
    user_encoder = LabelEncoder()
    product_encoder = LabelEncoder()
    
    df['user_idx'] = user_encoder.fit_transform(df['user_id'])
    df['product_idx'] = product_encoder.fit_transform(df['product_id'])
    
    # Tạo từ điển ánh xạ ngược để hiển thị kết quả
    idx_to_product = {idx: product for idx, product in zip(df['product_idx'], df['product_id'])}
    product_to_name = {pid: name for pid, name in zip(df['product_id'], df['product_name'])}
    
    # Tính trọng số cho các giá trị rating
    rating_counts = Counter(df['rating'])
    total_samples = len(df)
    num_classes = len(rating_counts)
    class_weights = {rating: total_samples / (num_classes * count) 
                    for rating, count in rating_counts.items()}
    
    # Thống kê
    num_users = df['user_idx'].nunique()
    num_products = df['product_idx'].nunique()
    print(f"Số lượng người dùng: {num_users}")
    print(f"Số lượng sản phẩm: {num_products}")
    print(f"Số lượng đánh giá: {len(df)}")
    print("Phân phối rating trước khi xử lý:")
    print(df['rating'].value_counts().sort_index())
    print("Trọng số của các rating:", class_weights)
    
    return df, user_encoder, product_encoder, idx_to_product, product_to_name, scaler, class_weights

# ====== FEATURE ENGINEERING ======
def calculate_purchase_count_estimated(df):
    """Tính toán số lượng mua ước tính dựa trên các đặc trưng"""
    a, b, c, d = 5, 0.2, 0.01, 1
    df['purchase_count_estimated'] = (
        a * df['rating'] +
        b * df['discount_percentage'] +
        c * df['actual_price'] +
        d * df['review_count']
    )
    return df

# ====== OVERSAMPLING FOR MISSING RATING_COUNT ======
def fix_rating_count_with_oversampling(df):
    """Sửa các giá trị thiếu trong rating_count bằng oversampling"""
    df['rating_count'] = df['rating_count'].fillna(0)
    df['rating_count'] = pd.to_numeric(df['rating_count'], errors='coerce')

    missing_mask = df['rating_count'] == 0
    if not missing_mask.any():
        return df

    product_avg = df[~missing_mask].groupby('product_id')['rating_count'].mean()
    global_avg = df['rating_count'].mean()
    global_avg = int(global_avg)

    for idx in df[missing_mask].index:
        product_id = df.loc[idx, 'product_id']
        if product_id in product_avg:
            df.loc[idx, 'rating_count'] = product_avg[product_id]
        else:
            df.loc[idx, 'rating_count'] = global_avg

    noise = np.random.normal(0, 0.1, len(df[missing_mask]))
    df.loc[missing_mask, 'rating_count'] = (df.loc[missing_mask, 'rating_count'] * (1 + noise)).astype(int)
    df['rating_count'] = df['rating_count'].clip(0).astype(int)
    return df

# ====== CHECK NaN ======
def print_columns_with_nan(df, step_name):
    """Kiểm tra và in ra các cột có giá trị NaN"""
    nan_cols = df.columns[df.isna().any()].tolist()
    if nan_cols:
        print(f"\n{step_name} - Columns with NaN values:")
        for col in nan_cols:
            print(f"  {col}: {df[col].isna().sum()} NaNs")
    else:
        print(f"\n{step_name} - No NaN values found.")

# ====== DATA ANALYSIS PLOTS ======
def plot_analysis(df):
    """Tạo các biểu đồ phân tích dữ liệu"""
    if df is None or len(df) == 0:
        print("Không có dữ liệu để phân tích")
        return
        
    try:
        # Phân bố giá
        plt.figure(figsize=(12, 6))
        sns.histplot(df['discounted_price'], bins=50, kde=True, color='blue', label='Discounted Price')
        sns.histplot(df['actual_price'], bins=50, kde=True, color='red', label='Actual Price', alpha=0.5)
        plt.xlabel("Giá sản phẩm")
        plt.ylabel("Số lượng sản phẩm")
        plt.title("Phân bố giá giảm và giá thực tế")
        plt.legend()
        plt.savefig("price_distribution.png")
        plt.close()
        
        # Phân bố số lượt đánh giá
        plt.figure(figsize=(10, 6))
        sns.histplot(df['rating_count'], bins=50, kde=True, color='purple')
        plt.xlabel("Số lượt đánh giá")
        plt.ylabel("Số lượng sản phẩm")
        plt.title("Phân bố số lượt đánh giá")
        plt.xscale('log')  # Dùng log scale nếu dữ liệu phân bố không đều
        plt.savefig("rating_count_distribution.png")
        plt.close()
        
        # Phân bố điểm đánh giá
        plt.figure(figsize=(10, 6))
        sns.histplot(df['rating'], bins=20, kde=True, color='green')
        plt.xlabel("Điểm đánh giá")
        plt.ylabel("Số lượng sản phẩm")
        plt.title("Phân bố điểm đánh giá")
        plt.savefig("rating_distribution.png")
        plt.close()
        
        print("\nBiểu đồ phân tích đã được lưu:")
        print("  - price_distribution.png")
        print("  - rating_count_distribution.png")
        print("  - rating_distribution.png")
    
    except Exception as e:
        print(f"Error creating plots: {str(e)}")
        print(f"Traceback: {traceback.format_exc()}")

# ====== MAIN PIPELINE ======
def main():
    try:
        # Đọc file CSV với các kiểu encoding khác nhau
        df = read_csv_with_multiple_encodings("amazon.csv")
        
        print(f"Loaded data: {df.shape[0]} rows, {df.shape[1]} columns")
        
        # Kiểm tra thông tin cơ bản
        print("\nKiểm tra dữ liệu ban đầu:")
        print(df.info())
        print("\nMẫu dữ liệu:")
        print(df.head())
        
        # Kiểm tra các cột có NaN
        print_columns_with_nan(df, "Initial Load")
    except Exception as e:
        print(f"Error loading file: {str(e)}")
        print(f"Traceback: {traceback.format_exc()}")
        return None, None

    # Drop unused columns
    columns_to_drop = ['about_product', 'review_content', 'user_name', 'review_title', 'product_link', 'img_link']
    df = df.drop(columns=[col for col in columns_to_drop if col in df.columns], errors='ignore')

    # Data Cleaning
    df['discounted_price'] = df['discounted_price'].apply(clean_price)
    df['actual_price'] = df['actual_price'].apply(clean_price)
    df['discount_percentage'] = df['discount_percentage'].apply(clean_percentage)
    df['rating_count'] = df['rating_count'].apply(clean_rating_count)
    df['rating'] = pd.to_numeric(df['rating'], errors='coerce').fillna(0)

    print_columns_with_nan(df, "After Cleaning Prices, Discounts & Ratings")

    # Preprocess data (encoding, rating scaling, and class weights)
    df, user_encoder, product_encoder, idx_to_product, product_to_name, rating_scaler, class_weights = preprocess_data(df)
    print_columns_with_nan(df, "After Preprocessing")

    # Create review_count
    if 'review_id' in df.columns:
        df['review_count'] = df.groupby('product_id')['review_id'].transform('count')
    else:
        df['review_count'] = 0
    df['review_count'] = pd.to_numeric(df['review_count'], errors='coerce').fillna(0).astype(int)

    # Fix missing rating_count
    df = fix_rating_count_with_oversampling(df)
    print_columns_with_nan(df, "After Fixing Missing rating_count")

    # Normalize numeric columns
    df = normalize_numeric(df)

    # Round discounted_price
    df['discounted_price'] = df['discounted_price'].round(1)

    # Estimate purchase count
    df = calculate_purchase_count_estimated(df)
    print_columns_with_nan(df, "After Calculating Purchase Count")

    # Handle outliers
    outlier_columns = [
        'rating', 'rating_count', 'discounted_price',
        'actual_price', 'discount_percentage', 'purchase_count_estimated'
    ]
    df = handle_outliers(df, outlier_columns)
    print_columns_with_nan(df, "After Handling Outliers")

    # Select final columns (bao gồm các cột mới từ preprocess_data)
    final_columns = [
        'product_id', 'product_name', 'category', 'review_id',
        'rating', 'rating_count', 'review_count',
        'discounted_price', 'actual_price', 'discount_percentage',
        'purchase_count_estimated', 'user_id', 'user_idx', 'product_idx', 'scaled_rating'
    ]
    
    # Kiểm tra và thêm các cột bổ sung nếu có
    additional_columns = ['Search_Frequency', 'Timestamp', 'Generated_Keywords']
    for col in additional_columns:
        if col in df.columns:
            final_columns.append(col)
    
    # Lọc các cột có trong DataFrame
    existing_columns = [col for col in final_columns if col in df.columns]
    df = df[existing_columns]

    # Round numeric columns
    columns_to_round = ['rating', 'rating_count', 'review_count', 'discounted_price', 
                        'actual_price', 'discount_percentage', 'purchase_count_estimated', 'scaled_rating']
    for col in columns_to_round:
        if col in df.columns:
            df[col] = df[col].round(1)

    # Save cleaned data
    try:
        df.to_csv("amazon_cleaned.csv", index=False)
        print("\nCleaned data saved to amazon_cleaned.csv")
        print(f"Final shape: {df.shape[0]} rows, {df.shape[1]} columns")
        print("\nFinal columns:")
        for col in df.columns:
            print(f"  - {col}")
        
        # Lưu class_weights để sử dụng sau này
        import json
        with open("class_weights.json", "w") as f:
            json.dump(class_weights, f)
        print("Class weights saved to class_weights.json")
        
        # Phân tích dữ liệu đã xử lý
        plot_analysis(df)
        return df, class_weights
        
    except Exception as e:
        print(f"Error saving file: {str(e)}")
        print(f"Traceback: {traceback.format_exc()}")
        return None, None

if __name__ == "__main__":
    df_processed, class_weights = main()

Đang thử đọc với encoding: utf-8
Đọc thành công với encoding: utf-8
Loaded data: 599137 rows, 25 columns

Kiểm tra dữ liệu ban đầu:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 599137 entries, 0 to 599136
Data columns (total 25 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   product_id           599137 non-null  object 
 1   product_name         599136 non-null  object 
 2   category             599137 non-null  object 
 3   discounted_price     599137 non-null  object 
 4   actual_price         599137 non-null  object 
 5   discount_percentage  599137 non-null  object 
 6   rating               599137 non-null  float64
 7   rating_count         599137 non-null  int64  
 8   about_product        599117 non-null  object 
 9   user_name            599137 non-null  object 
 10  review_id            599137 non-null  object 
 11  review_title         599087 non-null  object 
 12  review_content       599133 non-null  

In [2]:
import pandas as pd
from sklearn.utils import resample

# Đọc dữ liệu
df = pd.read_csv("amazon_cleaned.csv")

# Kiểm tra phân bố rating hiện tại
print(df['rating'].value_counts())

# Xác định số lượng nhỏ nhất trong các rating để cân bằng
min_count = df['rating'].value_counts().min()

# Cân bằng từng lớp rating
balanced_df = pd.concat([
    resample(df[df['rating'] == r], replace=False, n_samples=min_count, random_state=42)
    for r in sorted(df['rating'].unique())
])

# Shuffle lại dữ liệu
balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Kiểm tra lại phân bố
print("\nPhân bố sau cân bằng:")
print(balanced_df['rating'].value_counts())

# Lưu kết quả
balanced_df.to_csv("amazon_rating_balanced.csv", index=False)

rating
4.5    51700
4.4    48097
4.3    44363
4.6    43767
4.2    41144
4.0    38887
5.0    38412
4.1    37206
4.7    31129
3.9    29121
3.8    26361
3.7    23611
3.6    20378
2.7    19143
3.5    18654
4.8    15375
3.4    15119
3.3    13433
3.0    12138
3.2    10394
3.1     8482
4.9     4567
2.9     4358
2.8     3298
Name: count, dtype: int64

Phân bố sau cân bằng:
rating
4.8    3298
4.3    3298
4.2    3298
4.1    3298
4.5    3298
3.2    3298
4.0    3298
3.8    3298
3.4    3298
4.6    3298
3.7    3298
2.8    3298
4.7    3298
3.3    3298
3.6    3298
3.0    3298
4.4    3298
5.0    3298
3.1    3298
2.9    3298
3.5    3298
4.9    3298
3.9    3298
2.7    3298
Name: count, dtype: int64
